<a href="https://colab.research.google.com/github/KoreTane/MKTBank/blob/main/MKTBank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=1210a46b8a768987a9a1d52f21709ee763077a0e2b862eb001b520d3d2668d0a
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


# Exploratory Analysis


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col, count, when
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

spark = SparkSession.builder.appName("BankMarketing").getOrCreate()
df = spark.read.csv("bank.csv", header=True, inferSchema=True)
# Criar um ID único para cada cliente
df = df.withColumn("client_id", monotonically_increasing_id())

colunas = ['client_id'] + [col for col in df.columns if col != 'client_id']
df = df.select(colunas)

df.show(50)

df.printSchema()

df.summary().show()

+---------+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|client_id|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---------+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|        0| 59|      admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
|        1| 56|      admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
|        2| 41|  technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
|        3| 55|    services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| un

In [4]:
# Criar um ID único para cada cliente
df = df.withColumn("client_id", monotonically_increasing_id())

df.show(5)


+---------+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|client_id|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---------+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|        0| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
|        1| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
|        2| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
|        3| 55|  services|married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
|     

In [5]:
total_count = df.count()

distinct_count = df.distinct().count()

# Verificar se há duplicatas
if total_count > distinct_count:
    print("Existem dados duplicados no DataFrame.")
else:
    print("Não existem dados duplicados no DataFrame.")

Não existem dados duplicados no DataFrame.


In [22]:
# Análises Simples
# 1. age: Contar quantas IDs têm mais de 80 anos
age_above_80 = df.filter(df.age > 80).count()

# 2. Contando IDs agrupados por job e calculando a média do balance
job_count = df.groupBy("job") \
               .agg(
                   F.count("*").alias("count"),  # Contagem de IDs
                   F.format_number(F.avg("balance"), 2).alias("average_balance")  # Média do balance formatada
               ) \
               .sort(F.col("count").desc())

# 3. marital: Contar IDs agrupados por estado civil
marital_count = df.groupBy("marital").count().sort(col("count").desc())

# 4. Contando IDs agrupados por nível de escolaridade e calculando a média do balance
education_count = df.groupBy("education") \
                     .agg(
                         F.count("*").alias("count"),  # Contagem de IDs
                         F.format_number(F.avg("balance"), 2).alias("average_balance")  # Média do balance formatada
                     ) \
                     .sort(F.col("count").desc())

# 5. default: Contar IDs com e sem crédito em atraso
default_count = df.groupBy("default").count()

# 6. balance: Contar quantas IDs estão com balance negativo e positivo
balance_count = df.groupBy(F.when(df.balance < 0, "Negativo").otherwise("Positivo").alias("Balance")).count()

# 7. housing: Contar quantos IDs possuem housing e quantos não
housing_count = df.groupBy("housing").count()

# 8. loan: Contar quantos IDs possuem loan e quantos não
loan_count = df.groupBy("loan").count()

# 9. contact: Contagem do número de IDs agrupada por contact
contact_count = df.groupBy("contact").count()

# 10. day: Contar a distribuição de contatos por dia do mês
day_count = df.groupBy("day").count().sort(col("count").desc())

# 11. month: Contar a distribuição de contatos por mês
month_count = df.groupBy("month").count().sort(col("count").desc())

# 12. duration: Contar quantas IDs têm duração de contato acima e abaixo da média
average_duration = df.select(F.avg("duration")).first()[0]
duration_count = df.groupBy(F.when(df.duration < average_duration, "Abaixo da Média").otherwise("Acima da Média").alias("Duração")).count()

# 13. campaign: Distribuição de contatos feitos durante a campanha
campaign_count = df.groupBy("campaign").count()

# 14. pdays: Distribuição do número de dias desde o último contato (999 significa que nunca foi contatado)
pdays_count = df.groupBy(F.when(df.pdays == 999, "Nunca contatado").otherwise(df.pdays)).count()

# 15. previous: Distribuição do número de contatos anteriores
previous_count = df.groupBy("previous").count()

# 16. poutcome: Contar resultados da campanha anterior
poutcome_count = df.groupBy("poutcome").count()

# 17. deposit: Contagem de aceitação da oferta (sim/não)
deposit_count = df.groupBy("deposit").count()

# Exibindo os resultados em uma matriz

print("Análise Completa de Clientes:")

print(f"\n1. IDs com mais de 80 anos: {age_above_80}")

print("\n2. Contagem de IDs agrupados por Job:")
job_count.show()

print("\n3. Contagem de IDs agrupados por Estado Civil:")
marital_count.show()

print("\n4. Contagem de IDs agrupados por Nível de Escolaridade:")
education_count.show()

print("\n5. Contagem de IDs com e sem Crédito em Atraso:")
default_count.show()

print("\n6. Contagem de IDs com Balance Negativo e Positivo:")
balance_count.show()

print("\n7. Contagem de IDs com Housing:")
housing_count.show()

print("\n8. Contagem de IDs com Loan:")
loan_count.show()

print("\n9. Contagem de IDs agrupados por Contact:")
contact_count.show()

print("\n10. Distribuição de contatos por Dia do Mês:")
day_count.show()

print("\n11. Distribuição de contatos por Mês:")
month_count.show()

print("\n12. Duração do contato (Abaixo/Acima da Média):")
duration_count.show()

print("\n13. Distribuição de contatos feitos durante a Campanha:")
campaign_count.show()

print("\n14. Distribuição do Número de Dias desde o Último Contato:")
pdays_count.show()

print("\n15. Distribuição do Número de Contatos Anteriores:")
previous_count.show()

print("\n16. Resultados da Campanha Anterior:")
poutcome_count.show()

print("\n17. Aceitação da Oferta (Sim/Não):")
deposit_count.show()

Análise Completa de Clientes:

1. IDs com mais de 80 anos: 55

2. Contagem de IDs agrupados por Job:
+-------------+-----+---------------+
|          job|count|average_balance|
+-------------+-----+---------------+
|   management| 2566|       1,793.66|
|  blue-collar| 1944|       1,203.93|
|   technician| 1823|       1,556.29|
|       admin.| 1334|       1,195.87|
|     services|  923|       1,081.17|
|      retired|  778|       2,417.25|
|self-employed|  405|       1,865.37|
|      student|  360|       1,500.78|
|   unemployed|  357|       1,314.72|
| entrepreneur|  328|       1,621.94|
|    housemaid|  274|       1,366.16|
|      unknown|   70|       1,945.46|
+-------------+-----+---------------+


3. Contagem de IDs agrupados por Estado Civil:
+--------+-----+
| marital|count|
+--------+-----+
| married| 6351|
|  single| 3518|
|divorced| 1293|
+--------+-----+


4. Contagem de IDs agrupados por Nível de Escolaridade:
+---------+-----+---------------+
|education|count|average_balanc

In [15]:
# Filtrar os registros onde job é igual a "unknown"
unknown_job_df = df.filter(df.job == "unknown")

# Exibir os resultados
unknown_job_df.show(80)

+---------+---+-------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+-------+
|client_id|age|    job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---------+---+-------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+-------+
|       66| 49|unknown| married|  primary|     no|    341|    yes| yes|  unknown| 15|  may|     520|       2|   -1|       0| unknown|    yes|
|      668| 52|unknown| married|  unknown|     no|   5361|     no|  no| cellular|  5|  aug|     607|       3|   -1|       0| unknown|    yes|
|      861| 60|unknown| married|  unknown|     no|  17297|     no|  no| cellular| 26|  aug|     664|      11|   -1|       0| unknown|    yes|
|     1075| 31|unknown| married|secondary|     no|    111|     no|  no| cellular| 21|  nov|     504|       2|   93|       2| failure|    yes|
|     